In [11]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [15]:
#Get the spam data collection 
df = pd.read_csv("Spam.csv")
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Go until jurong point, crazy.. Available only ..."
5,ham,Ok lar... Joking wif u oni...
6,spam,Free entry in 2 a wkly comp to win FA Cup fina...
7,ham,U dun say so early hor... U c already then say...


In [16]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Go until jurong point, crazy.. Available only ..."


In [21]:
df.describe()

df.columns = ['response','message']

df

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Go until jurong point, crazy.. Available only ..."
5,ham,Ok lar... Joking wif u oni...
6,spam,Free entry in 2 a wkly comp to win FA Cup fina...
7,ham,U dun say so early hor... U c already then say...


In [22]:
#view response 
df.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham            6      3  U dun say so early hor... U c already then say...   
spam           2      1  Free entry in 2 a wkly comp to win FA Cup fina...   

               
         freq  
response       
ham         2  
spam        2

In [23]:
#Verify length of the messages and also add it as a new column 
df['length'] = df['message'].apply(len)

In [24]:
df.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Go until jurong point, crazy.. Available only ...",111


In [28]:
#define a function to get rid of stopwords present in the messages
def nowords(mess):
    m = [char for char in mess if char not in string.punctuation]
    
    message = ''.join(m)
    
    return [word for word in message if word.lower() not in stopwords.words('english')]




In [30]:
df['message'].head(5).apply(nowords)

0    [G,  , u, n, l,  , j, u, r, n, g,  , p, n,  , ...
1     [k,  , l, r,  , J, k, n, g,  , w, f,  , u,  , n]
2    [F, r, e, e,  , e, n, r,  , n,  , 2,  ,  , w, ...
3    [U,  , u, n,  ,  ,  , e, r, l,  , h, r,  , U, ...
4    [G,  , u, n, l,  , j, u, r, n, g,  , p, n,  , ...
Name: message, dtype: object

In [34]:
#start text processing with vectorizer 
from  sklearn.feature_extraction.text import CountVectorizer

In [64]:
#use bag of words by applying the function and fit the data into it
bow = CountVectorizer(analyzer = nowords).fit(df['message'])

In [46]:
#print length of bag of words stored in the vocabulary_ attribute
print(len(bow.vocabulary_))

31


In [47]:

bow1 = bow.transform(df['message'])

In [55]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer

bow2 = TfidfTransformer().fit(bow1)

In [57]:
#print shape of the tfidf 
bow3 = bow2.transform(bow1)
bow3.shape

(8, 31)

In [74]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detect_model  = MultinomialNB().fit(bow3,df['response'])

In [78]:
#check model for the predicted and expected value say for message#2 and message#5
message = df['message'][1]

bag_of_words = bow.transform([message])

tfidf_transformer  = bow2.transform(bag_of_words)

print("predict resonse",spam_detect_model.predict(tfidf_transformer)[0])
print("expected response",df.response[1])

predict resonse ham
expected response ham


In [76]:
message = df['message'][4]

bag_of_words = bow.transform([message])

tfidf_transformer =bow2.transform(bag_of_words)

print("predicted",spam_detect_model.predict(tfidf_transformer)[0])

print("expected response",df.response[4])

predicted ham
expected response ham
